<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-gliner-multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/r-kovalch/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 420 (delta 105), reused 101 (delta 41), pack-reused 231 (from 1)
Receiving objects: 100% (420/420), 2.56 MiB | 27.54 MiB/s, done.
Resolving deltas: 100% (259/259), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 16.24 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [5]:
ls

combine_corpora-en-fr-nl.sh  dataset_processor.py        train_full.tsv
combine_corpora-en-fr.sh     output/                     train_model.py
combine_corpora_gliner.sh    preprocess_acter_gliner.py  train_spacy_model.py
combine_corpora.sh           preprocess_acter.py         Untitled.ipynb
configs/                     test_full.tsv


In [6]:
!bash combine_corpora_gliner_multi.sh

Writing JSON for GLiNER


In [7]:
!cp train_full.tsv train_full.jsonl && \
  cp test_full.tsv test_full.jsonl && \
  cp val_full.tsv val_full.jsonl

cp: cannot stat 'val_full.tsv': No such file or directory


In [8]:
!pip install -U "gliner>=0.2.19" "transformers>=4.51.0" \
               datasets accelerate evaluate seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
 

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from pathlib import Path
from datasets import load_dataset
from gliner import GLiNER
from gliner.data_processing import GLiNERDataset, WordsSplitter
from gliner.data_processing.collator import DataCollatorWithPadding
from gliner.training import Trainer, TrainingArguments

import torch
from transformers import EarlyStoppingCallback

In [19]:
# 2. Load ACTER JSONL
raw = load_dataset(
    "json",
    data_files={"train": "./train_full.jsonl",
                "validation": "./test_full.jsonl"},
)

# 3. Char→token conversion (unchanged)
LABEL_SET=set()
def to_gliner(ex):
    tokens, c2t, p = [], {}, 0
    for i, tok in enumerate(ex["text"].split()):
        tokens.append(tok); c2t.update({p+j:i for j in range(len(tok))}); p+=len(tok)+1
    ner=[]
    for ent in ex["entities"]:
        s,e=c2t.get(ent["start"]),c2t.get(ent["end"]-1)
        if s is not None and e is not None:
            lbl=ent["label"].lower(); ner.append([s,e,lbl]); LABEL_SET.add(lbl)
    item={"tokenized_text":tokens,"ner":ner}
    if not ner: item["label"]=sorted(LABEL_SET)
    return item
train_py=[to_gliner(x) for x in raw["train"]]
dev_py  =[to_gliner(x) for x in raw["validation"]]
labels=sorted(LABEL_SET)

# 4. Dataset
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
if hasattr(model.transformer_model, "gradient_checkpointing_enable"):
    model.transformer_model.gradient_checkpointing_enable()

collator = DataCollatorWithPadding(model.config)
tok   = model.data_processor.transformer_tokenizer

splitter = WordsSplitter(model.config.words_splitter_type)
train_ds = GLiNERDataset(train_py, model.config, tok, splitter, entities=labels)
dev_ds   = GLiNERDataset(dev_py,  model.config, tok, splitter, entities=labels)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ucu/ner/gliner_multi",
    learning_rate=5e-6,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=35,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,        # ← keep checkpoint with lowest loss
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    eval_accumulation_steps=1,          # stream logits → CPU, prevents OOM
    fp16=True,
    seed=42,
    report_to="none",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=collator,
    # NOTE: no compute_metrics => just loss is computed
    preprocess_logits_for_metrics=lambda l, _: l.float().cpu(),  # free GPU mem
)
trainer.train()

<ipython-input-20-994eb828d9f2>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: The size of tensor a (6) must match the size of tensor b (29) at non-singleton dimension 1

In [ ]:
# path to the best-loss checkpoint
best_path = "/content/drive/MyDrive/ucu/ner/nuner_acter_ft/"
print("Best checkpoint:", best_path)
best_model = GLiNER.from_pretrained(best_path).to("cuda")
out_str, f1 = best_model.evaluate(
    dev_py,
    threshold=0.35,
    entity_types=labels,
    batch_size=1         # adjust until it fits
)
print(out_str)          # P: 65.06%	R: 88.52%	F1: 75.00%


In [ ]:
# from google.colab import runtime
# runtime.unassign()